In [2]:
import boto3
from io import BytesIO
import pandas as pd 
import requests 
import datetime as dt 
from secrets_config import api_key 

In [4]:
# use requests to get the current weather for your city 
city_name = "perth"
response = requests.get(f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}&units=metric")
if response.status_code == 200: 
    data = response.json()

In [7]:
# flatten json data into a pandas dataframe 
df = pd.json_normalize(data)
df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,wind.speed,wind.deg,rain.1h,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 500, 'main': 'Rain', 'description': 'l...",stations,10000,1660047604,28800,2063523,Perth,200,115.8333,-31.9333,...,92,2.06,200,0.37,0,2,63154,AU,1659999553,1660038320


In [17]:
# write data to an S3 bucket using the parquet file format 
# append the current timestamp to the filepath 
s3 = boto3.resource("s3")
bucket = s3.Bucket("dec-001")
current_timestamp = dt.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
key = f"weather/perth_{current_timestamp}"
out_buffer = BytesIO()
df.to_parquet(out_buffer, index=False)
bucket.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='dec-001', key='weather/perth_2022-08-09T20:33:09')